First function in MFE Practical Work 2 assignment

Designed to take a daily price pandas series

Returns a pandas dataframe of mean, std_dev, skew, kurt: 

Sampled from daily, weekly, monthly and annual data

In [7]:
import numpy as np
import pandas as pd

In [143]:
def summary_statistics(prices):

    # Sample data into daily, weekly, monthly and annual prices
    d_prices = prices
    w_prices = prices.resample("W-FRI").last()
    m_prices = prices.resample("M").last()
    q_prices = prices.resample("Q").last()

    # Convert prices to returns
    d_returns = d_prices.pct_change()
    w_returns = w_prices.pct_change()
    m_returns = m_prices.pct_change()
    q_returns = q_prices.pct_change()

    # Enclosed function to find mean, std, skew and kurt: pre - annualised 
    def process(returns, n):
        returns_mu = returns.mean()                         # mean returns per time period
        annual_mu = returns_mu * n                          # annualised mean returns

        returns_err = returns - returns_mu       
        returns_var = (returns_err**2).mean()               # average squared return ( 2nd moment)
        annual_var = returns_var * n                        # annual rescaled variance
        annual_std = np.sqrt(annual_var)                    # annual rescaled std deviation

        returns_mom3 = (returns_err**3).mean()
        returns_mom4 = (returns_err**4).mean()              # Third and Fourth moments


        annual_skew = (returns_mom3 / returns_var ** (3/2)) * np.sqrt(n)
        annual_kurt = (returns_mom4 / returns_var ** (4/2)) * n

        x_stats = pd.Series([annual_mu, annual_std, annual_skew, annual_kurt], index = ["mean", "std", "skew", "kurt"])
        
        return x_stats
         
    # Feed in daily, weekly, monthly and quarterly data
    d_stats = process(d_returns, 252).rename("Daily")
    w_stats = process(w_returns, 52 ).rename("Weekly")
    m_stats = process(m_returns, 12 ).rename("Monthly")
    q_stats = process(q_returns, 4  ).rename("Quarterly")

    # Concatenate the series
    stats = pd.concat([d_stats, w_stats, m_stats, q_stats], axis=1,)

    return stats

In [187]:
aapl = pd.read_csv("data/aapl.csv", parse_dates=True, index_col="Date")
aapl_close = aapl["Close"]
sbry = pd.read_csv("data/SBRY.L.csv", parse_dates=True, index_col="Date")
sbry_close = sbry["Close"]

In [188]:
summary_statistics(aapl_close)

,Daily,Weekly,Monthly,Quarterly
mean,0.285387,0.305630,0.319264,0.316689
std,0.243743,0.243948,0.266882,0.296066
skew,-4.757964,-0.429231,-1.844991,-1.786075
kurt,1926.154100,263.561144,39.191903,14.764489


In [190]:
summary_statistics(sbry_close)

,Daily,Weekly,Monthly,Quarterly
mean,-0.059793,-0.057119,-0.164591,0.025990
std,0.308372,0.298536,0.301015,0.251771
skew,-54.359123,-8.552671,-1.465545,-1.212365
kurt,8692.463139,451.358353,32.307952,6.000000
